In [ ]:
import os
import pickle
import pandas as pd

import ipywidgets as widgets
from ipywidgets import interact

import plotly.express as px
import plotly.graph_objects as go

from src.data_load import load_tables, load_artificial_instance, load_distances
from src.metrics import  collect_results_to_df, compute_metrics_with_moves
from src.alpha_tuning_utils import compute_baseline_metrics, _collect_alpha_metrics, \
    add_aggregated_totals, fechas, alphas, iterations_nums

data_path = '../data'
instance = 'inst3'


directorio_df, labors_raw_df, cities_df, duraciones_df, valid_cities = load_tables(data_path, generate_labors=False)
labors_real_df = load_artificial_instance(data_path, instance, labors_raw_df)
dist_dict = load_distances(data_path, 'real', instance)

# Description

This notebook provides an analysis of the experimentation of alpha calibration with a greedy algorithm.

# Process results data

In [ ]:
baseline_df = compute_baseline_metrics(
    data_path=data_path, 
    instance=instance, 
    fechas=fechas, 
    valid_cities=valid_cities, 
    dist_dict=dist_dict
)

metrics_df = _collect_alpha_metrics(
    data_path=data_path,
    alphas=alphas,
    iterations_nums=iterations_nums,
)

metrics_df = metrics_df.join(baseline_df.set_index("city"), on="city")
metrics_w_totals_df = add_aggregated_totals(metrics_df)

# Variando el alpha

In [ ]:
def plot_metrics_vs_iterations(metrics_df):
    def _plot(alpha, city, metric):
        # Filtrar por alpha y city
        df = metrics_df[(metrics_df["alpha"] == alpha) & (metrics_df["city"] == city)]
        if df.empty:
            print("⚠️ No data for this selection")
            return

        # Crear figura principal para el algoritmo
        fig = px.line(
            df,
            x="num_iterations",
            y=metric,
            title=f"alpha={alpha}, city={city}",
            markers=True
        )

        # --- Añadir baseline como serie scatter ---
        baseline_col = f"{metric}_baseline"
        if baseline_col in df.columns:
            baseline_value = df[baseline_col].iloc[0]  # Baseline constante
            fig.add_trace(
                go.Scatter(
                    x=df["num_iterations"],
                    y=[baseline_value] * len(df),
                    mode="lines",
                    line=dict(color="red", dash="dash"),
                    name="Baseline"
                )
            )

        # --- Layout ---
        fig.update_layout(
            xaxis_title="Number of Iterations",
            yaxis_title=metric,
            template="plotly_white",
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=-0.3,
                xanchor="center",
                x=0.5
            )
        )

        # Renombrar el trazo principal para la leyenda
        fig.update_traces(name="Algorithm", selector=dict(mode="markers+lines"))

        fig.show()

    interact(
        _plot,
        alpha=widgets.Dropdown(
            options=sorted(metrics_df["alpha"].unique()),
            description="Alpha:"
        ),
        city=widgets.Dropdown(
            options=sorted(metrics_df["city"].unique()),
            description="City:"
        ),
        metric=widgets.Dropdown(
            options=["vt_labors", "tiempo_extra", "driver_move_distance"],
            description="Metric:"
        )
    )


In [ ]:
plot_metrics_vs_iterations(metrics_w_totals_df)

interactive(children=(Dropdown(description='Alpha:', options=(np.float64(0.0), np.float64(0.1), np.float64(0.2…

# Variando el número de iteraciones

In [ ]:
def plot_metrics_vs_alpha(metrics_df):
    def _plot(num_iter, city, metric):
        # Filtrar por iteración y ciudad
        df = metrics_df[(metrics_df["num_iterations"] == num_iter) & (metrics_df["city"] == city)]
        if df.empty:
            print("⚠️ No data for this selection")
            return

        # Crear figura principal para el algoritmo
        fig = px.line(
            df,
            x="alpha",
            y=metric,
            title=f"{metric} vs. Alpha<br>iterations={num_iter}, city={city}",
            markers=True
        )

        # --- Añadir baseline como serie scatter ---
        baseline_col = f"{metric}_baseline"
        if baseline_col in df.columns:
            baseline_value = df[baseline_col].iloc[0]  # Baseline constante
            fig.add_trace(
                go.Scatter(
                    x=df["alpha"],
                    y=[baseline_value] * len(df),
                    mode="lines",
                    line=dict(color="red", dash="dash"),
                    name="Baseline"
                )
            )

        # --- Layout ---
        fig.update_layout(
            xaxis_title="Alpha",
            yaxis_title=metric,
            template="plotly_white",
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=-0.3,
                xanchor="center",
                x=0.5
            )
        )

        # Renombrar el trazo principal para la leyenda
        fig.update_traces(name="Algorithm", selector=dict(mode="markers+lines"))

        fig.show()

    interact(
        _plot,
        num_iter=widgets.Dropdown(
            options=sorted(metrics_df["num_iterations"].unique()),
            description="Iterations:"
        ),
        city=widgets.Dropdown(
            options=sorted(metrics_df["city"].unique()),
            description="City:"
        ),
        metric=widgets.Dropdown(
            options=["vt_labors", "tiempo_extra", "driver_move_distance"],
            description="Metric:"
        )
    )


In [ ]:
plot_metrics_vs_alpha(metrics_w_totals_df)

interactive(children=(Dropdown(description='Iterations:', options=(np.int64(10), np.int64(25), np.int64(50), n…